 # Milestone 3: Training a Supervised Image Classifier on the Melanoma Dataset using Transfer Learning

 In machine learning, there are no ready-to-use recipes. Certain approaches have been found to work well for certain cases, but in the end, much of it comes down to trial and error. [Occam’s razor](https://en.wikipedia.org/wiki/Occam%27s_razor) (the principle roughly summarized as “the simplest solution is usually the best one”) suggests starting from the most basic method, and building up on top of it until we get the performance we want. Keeping track of improvements we get at each step also helps us justify our efforts!

 ## Part A: The Data

 Here I reuse the Dataset code that I wrote for Milestone 2.

 Even in the extreme situation of only having 200 labeled samples available for training, we still have to set aside some number of them for the validation set. The fewer training samples we have, the greater the risk of overfitting, which makes having a validation set absolutely essential. It is up to you to decide what fraction of the precious few labeled samples to keep for validation purposes; I set it at 30% or 60 images, as anything smaller than that would likely result in a major val set overfitting problem.

In [ ]:
import os
from os import listdir
from os.path import join

from PIL import Image
import random

import matplotlib.pyplot as plt


import numpy as np
import copy

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import  models
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Device: ", device)

from pathlib import Path

In [ ]:
%matplotlib inline

In [ ]:
# Define a function to set the seed. This will make results reproducible.
def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
            
    # [your code here]

# Define a function to check if a file is actually an image.
def is_image(filename):
    if isinstance(filename, str):
        filename = Path(filename)
    return filename.suffix in ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']

In [ ]:
# Some test for Path 

dir_labeled = "./data/MelanomaDetectionLabeled/labeled/"
dir_test  = "./data/MelanomaDetectionLabeled/test/"

In [ ]:
class LabeledDataset(Dataset):

    def __init__(self, dir_path, transform=None):
        """
        Args:
            dir_path (string): Directory containing the images.
            transform (optional): Optional transform to be applied
                on an image.
        """
        
        # The list of all the image file names (but not the images themselves!) will be read
        # when the Dataset object is initialized
        p = Path(dir_path).resolve()
        self.img_path_list = [f for f in p.iterdir() if is_image(f)]
        self.transform = transform
        
        
    def __len__(self):
        return len(self.img_path_list)    

    def __getitem__(self, idx):
        
        # Here is where the image actually gets read:
        img_path = self.img_path_list[idx]
        # img = read_image(str(img_path)).type(torch.float32)/255.0        
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
            
        img = np.asarray(img)
        label = int(img_path.stem.split('_')[1])

        return img, label
    
    
# test code 

labeled_set = LabeledDataset(dir_labeled)

print('Number of image is {}'.format(len(labeled_set)))
print('Label:', labeled_set[0][1])

labeled_set[100][0]

In [ ]:
labeled_loader = DataLoader(labeled_set, batch_size=4, shuffle=True)

for batch in labeled_loader:
    print(batch[0].shape)
    print(batch[1].shape)
    break

In [ ]:

for batch_idx, batch in enumerate(labeled_loader):    
    if batch_idx > 0:
        break 
    
    print("Batch labels", batch[1].data)
    image_grid = torchvision.utils.make_grid(batch[0], nrow=4)
    plt.imshow(image_grid.permute(1, 2, 0))
    plt.pause(0.05)

In [ ]:
# Choose some transformations
rotation = transforms.RandomChoice([transforms.RandomRotation([-3, 3]), 
                                    transforms.RandomRotation([87, 93]), 
                                    transforms.RandomRotation([177,183]),
                                    transforms.RandomRotation([267, 273])])
augmentation = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                   transforms.RandomVerticalFlip(), 
                                   rotation])

In [ ]:
# Instantiate the Label Dataset class for training
labeled_set = LabeledDataset(dir_labeled, transform=transforms.Compose([transforms.ToTensor(), augmentation]))

In [ ]:
# Randomly split the dataset (don't forget to set the seed!)
set_seed(123)

train_set, val_set = torch.utils.data.random_split(labeled_set, [0.7, 0.3])

In [ ]:
# Instantiate the LabeledDataset class for testing
test_set = LabeledDataset(dir_test, transform=transforms.ToTensor())

# Print the number of images in the train, validation and test sets
print("Number of images in the training set: ", len(train_set))
print("Number of images in the validation set: ", len(val_set))
print("Number of images in the test set: ", len(test_set))

# Write data loaders for training, validation and testing
# Since we have so few samples for the fully supervised part, you can use the entire set 
# at each training iteration, rather than separate it into mini-batches:

# [your code here]

In [ ]:
labeled_set = LabeledDataset(dir_labeled, transform=transforms.ToTensor())
img = labeled_set[0][0]
print(img)